<h1>Scraping movie data from cineb.com</h1>

In [1]:
# Imports
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import csv
import requests


In [2]:
# %% Setup version

print('pandas version: {}'.format(pd.__version__))
print('bs4 version: {}'.format(bs4.__version__))
print('requests version: {}'.format(requests.__version__))
print('csv version: {}'.format(csv.__version__))

pandas version: 1.4.2
bs4 version: 4.11.1
requests version: 2.27.1
csv version: 1.0


In [3]:
# genreate links 
base_url= 'https://cineb.net/movie?page={}'
pages= 1172
links= []
for i in range(pages):
    links.append(base_url.format(i))
links

['https://cineb.net/movie?page=0',
 'https://cineb.net/movie?page=1',
 'https://cineb.net/movie?page=2',
 'https://cineb.net/movie?page=3',
 'https://cineb.net/movie?page=4',
 'https://cineb.net/movie?page=5',
 'https://cineb.net/movie?page=6',
 'https://cineb.net/movie?page=7',
 'https://cineb.net/movie?page=8',
 'https://cineb.net/movie?page=9',
 'https://cineb.net/movie?page=10',
 'https://cineb.net/movie?page=11',
 'https://cineb.net/movie?page=12',
 'https://cineb.net/movie?page=13',
 'https://cineb.net/movie?page=14',
 'https://cineb.net/movie?page=15',
 'https://cineb.net/movie?page=16',
 'https://cineb.net/movie?page=17',
 'https://cineb.net/movie?page=18',
 'https://cineb.net/movie?page=19',
 'https://cineb.net/movie?page=20',
 'https://cineb.net/movie?page=21',
 'https://cineb.net/movie?page=22',
 'https://cineb.net/movie?page=23',
 'https://cineb.net/movie?page=24',
 'https://cineb.net/movie?page=25',
 'https://cineb.net/movie?page=26',
 'https://cineb.net/movie?page=27',
 '

In [4]:
# Prototype Scraping: Moneyball
url= 'https://cineb.net/watch-movie/watch-moneyball-free-18763.5349739'
page= requests.get(url)
soup= BeautifulSoup(page.content, 'html.parser')

# Find the div tag that contains the data to be scraped
div= soup.find('div', {'class': 'col-xl-6 col-lg-12 col-infor'})
div

<div class="col-xl-6 col-lg-12 col-infor">
<div class="dp-i-content">
<div class="dp-i-c-poster">
<div class="film-poster mb-2">
<img alt="Moneyball" class="film-poster-img" src="https://img.cineb.net/xxrz/250x400/188/b7/9c/b79c188b828a30f47601595fac1d13b2/b79c188b828a30f47601595fac1d13b2.jpg" title="Moneyball"/>
</div>
<div class="block-rating" id="block-rating"></div>
</div>
<div class="dp-i-c-right">
<h2 class="heading-name"><a href="/movie/moneyball-18763">Moneyball</a></h2>
<div class="dp-i-stats">
<span class="item mr-1"><button class="btn btn-sm btn-trailer" data-target="#modaltrailer" data-toggle="modal"><i class="material-icons mat-icon mr-2">videocam</i>Trailer</button></span>
<span class="item mr-1"><button class="btn btn-sm btn-quality"><strong>N/A</strong></button></span>
<span class="item mr-2"><button class="btn btn-sm btn-imdb">IMDB: 7.6</button></span>
</div>
<div class="description">
            The story of Oakland Athletics general manager Billy Beane's successful a

In [5]:
# Now let's scrape dara and assign it to variables
# first div: image
image_tag= div.find('img', {'class': 'film-poster-img'})
image_link= image_tag['src']

# Title
title= image_tag['title']

# Rating 
rating= div.find('button', {'class': 'btn btn-sm btn-imdb'}).text.strip('IMDB: ')

# Link
link= 'https://cineb.net'+ div.find('h2', {'class': 'heading-name'}).a['href']

# Description
description= div.find('div', {'class': 'description'}).text.strip('\n ')

# elements div 
elements_div= div.find('div', { 'class': 'elements'})
elements= elements_div.find_all('div', {'class': 'row-line'})

# Release date
release_date= elements[0].text.strip('\nReleased: ')

# Genre: I advise against using non-scalar values as it makes it hard to filter
# and clean data, but the goal here is to scrape as much data as we can.

genre_a_tags= elements[1].find_all('a')
if len(genre_a_tags) > 1:
    genre= []
    for genre_a_tag in genre_a_tags:
        genre.append(genre_a_tag.text)
else:
    genre= genre_a_tags[0].text
# Casts
casts=[]
a_tags= elements[2].find_all('a')
for a_tag in a_tags:
    casts.append(a_tag.text)
    
# Duration
duration= elements[3].text.strip('\nDuration: ').replace('\n                m','')

# Country
try:
    country= elements[4].a['title']
except:
    country= 'None'
    
# Production
production= []
a_tags= elements[5].find_all('a')
for a_tag in a_tags:
    production.append(a_tag.text)

# Movie data
movie= (title, rating, description, release_date, genre, casts, duration, country, production)
print(movie)


('Moneyball', '7.6', "The story of Oakland Athletics general manager Billy Beane's successful attempt to put together a baseball team on a budget, by employing computer-generated analysis to draft his players.", '2011-09-22', 'Drama', ['Brad Pitt', 'Jonah Hill', 'Philip Seymour Hoffman', 'Robin Wright', 'Chris Pratt'], '134', 'United States of America', ['Columbia Pictures', 'Scott Rudin Productions'])


In [6]:
country

'United States of America'

In [7]:
# Scrape movies link from a page
url= 'https://cineb.net/movie?page=816'
page= requests.get(url)
soup= BeautifulSoup(page.content, 'html.parser')

# Find the div tag that contains the data to be scraped
posters= soup.find_all('div', {'class': 'film-poster'})
m_links= []
for poster in posters:
    m_links.append(poster.a['href'])
m_links

['https://cineb.to/movie/watch-two-for-joy-free-14737',
 '/movie/watch-journey-back-to-christmas-free-14736',
 '/movie/watch-armadillo-free-14735',
 '/movie/watch-the-night-watchmen-free-14734',
 '/movie/watch-last-vermont-christmas-free-14733',
 '/movie/watch-the-walking-deceased-free-14731',
 '/movie/watch-the-hatton-garden-job-free-14729',
 '/movie/watch-last-dance-free-14726',
 '/movie/watch-crazy-kind-of-love-free-14722',
 'https://cineb.to/movie/watch-loose-cannons-free-14721',
 '/movie/watch-gangnam-blues-free-14720',
 '/movie/watch-feedback-free-14718',
 '/movie/watch-proxy-free-14717',
 'https://cineb.to/movie/watch-the-nun-free-14716',
 '/movie/watch-the-summer-of-sangaile-free-14714',
 '/movie/watch-fred-armisen-standup-for-drummers-free-14713',
 '/movie/watch-jack-goes-boating-free-14712',
 '/movie/watch-the-thief-of-bagdad-free-14711',
 '/movie/watch-nadine-free-14708',
 '/movie/watch-vip-free-14707',
 '/movie/watch-vagabond-free-14706',
 '/movie/watch-no-mercy-free-14705'

<h1>Now let's bring all this together</h1>

In [9]:

base_url= 'https://cineb.net/movie?page={}'

# The movies secion contains more than 1700 pages, but for this tutorial we'll scrape way less pages 
pages= 50
urls= []
for i in range(pages):
    urls.append(base_url.format(i))
    
records= []
for url in urls:

    page= requests.get(url)
    soup= BeautifulSoup(page.content, 'html.parser')

    # Find the div tag that contains the data to be scraped
    posters= soup.find_all('div', {'class': 'film-poster'})
    for poster in posters:

        m_link=(poster.a['href'])
        if 'cineb' in m_link:
            pass
        else:
            m_link= 'https://cineb.net'+ m_link
        page= requests.get(m_link)
        soup= BeautifulSoup(page.content, 'html.parser')

        # Find the div tag that contains the data to be scraped
        div= soup.find('div', {'class': 'col-xl-6 col-lg-12 col-infor'})

        # Image
        image_tag= div.find('img', {'class': 'film-poster-img'})
        image_link= image_tag['src']

        # Title
        title= image_tag['title']

        # Rating 
        rating= div.find('button', {'class': 'btn btn-sm btn-imdb'}).text.strip('IMDB: ')

        # Description
        description= div.find('div', {'class': 'description'}).text.strip('\n ')

        # elements div 
        elements_div= div.find('div', { 'class': 'elements'})
        elements= elements_div.find_all('div', {'class': 'row-line'})

        # Release date
        release_date= elements[0].text.strip('\nReleased: ')

        # Genre
        genre_a_tags= elements[1].find_all('a')
        if len(genre_a_tags) > 0:
            genre= genre_a_tags[0].text
        else:
            genr= 'Undfined'
            
        # Casts
        casts=[]
        a_tags= elements[2].find_all('a')
        for a_tag in a_tags:
            casts.append(a_tag.text)

        # Duration
        duration= elements[3].text.strip('\nDuration: ').replace('\n        m','')

        # Country
        try:
            country= elements[4].a['title']
        except:
            country= 'None'
        # Production
        production= []
        a_tags= elements[5].find_all('a')
        for a_tag in a_tags:
            production.append(a_tag.text)

        record= (title, rating, description, release_date, genre, casts, duration, country, production, m_link)
        records.append(record)
         

In [11]:
# Let's write those records in a csv file
with open("movies.csv", 'w', newline='', encoding='utf-8') as f:
    writer= csv.writer(f)
    writer.writerow(['title', 'rating', 'description', 'release_date', 'genre', 'casts', 'duration', 'country', 'production', 'm_link'])
    writer.writerows(records)

In [12]:
df= pd.read_csv('movies.csv')

In [13]:
df

,title,rating,description,release_date,genre,casts,duration,country,production,m_link
0,Where Are You,NaN,A photographer suffering under artist decline ...,2022-10-21,Thriller,"['Brad Greenquist', 'Madeline Brewer', 'Ray Ni...",94.0,United States of America,['Carte Blanche'],https://cineb.net/movie/watch-where-are-you-fr...
1,The Domestic,NaN,An upper-class couple hire the daughter of the...,2022-10-21,Thriller,['Amanda Du-Pont.'],110.0,South Africa,['Mandeville Films'],https://cineb.net/movie/watch-the-domestic-fre...
2,A Chance Encounter,NaN,"In the Sicilian town of Taormina, Italy, an as...",2022-10-28,Drama,"['Vincenzo Vivenzio', 'Kenny Burns', 'Jason Ed...",91.0,United States of America,['Bespoke Works'],https://cineb.net/movie/watch-a-chance-encount...
3,V/H/S/99,NaN,A teenager’s home video leads to a series of h...,2022-09-15,Horror,"['Jackson Kelly', 'Verona Blue', 'Archelaus Cr...",99.0,United States of America,"['Bloody Disgusting', 'Radio Silence', 'Studio...",https://cineb.to/movie/watch-vhs99-free-89401
4,Raymond & Ray,NaN,Half brothers Raymond and Ray reunite when the...,2022-10-14,Drama,"['Gina Jun', 'Sophie Okonedo', 'Vondie Curtis-...",106.0,United States of America,"['Esperanto Filmoj', 'Apple Inc.', 'Mockingbir...",https://cineb.net/movie/watch-raymond-ray-free...
...,...,...,...,...,...,...,...,...,...,...
1595,Lilith,6.8,"Vincent Bruce, a war veteran, begins working a...",1965-01-15,Drama,"['Warren Beatty', 'Kim Hunter', 'Anne Meacham'...",114.0,United States of America,"['Columbia Pictures', 'Centaur Enterprises']",https://cineb.net/movie/watch-lilith-free-83005
1596,Where the Scary Things Are,3.0,The horror begins as Ayla and her high-school ...,2022-06-28,Horror,"['Paul Cottman', 'Michael Cervantes', 'Selina ...",93.0,United States of America,['Lionsgate'],https://cineb.net/movie/watch-where-the-scary-...
1597,What The Nanny Saw,4.8,When nanny Kim accidentally discovers that her...,2022-04-29,Thriller,"['Lindsay Hartley', 'Ryan Francis', 'Laurie Fo...",85.0,United States of America,['Almost Never Films Inc'],https://cineb.net/movie/watch-what-the-nanny-s...
1598,Les Girls,6.6,After writing a tell-all book about her days i...,1957-10-03,Music,"['Gene Kelly', 'Jacques Bergerac', 'Mitzi Gayn...",114.0,United States of America,"['Sol C. Siegel Productions', 'Metro-Goldwyn-M...",https://cineb.net/movie/watch-les-girls-free-8...


In [14]:
df.shape

(1600, 10)